Authors: Louis Ravillon, Martin Piana
Date: October 2020
hours invested 1st week october: 4h

In [1]:
# packages

import requests
from bs4 import BeautifulSoup

In [2]:
x=requests.get("https://bib.cnrs.fr/home/").content
print(x)

b'<html>\n    <head>\n        <title>BibCnrs</title>\n        <meta charset="utf-8"/>\n        <meta name="description" content="Portail BibCnrs IST">\n        <meta name="apple-mobile-web-app-capable" content="yes" />\n        <meta name="robots" content="index follow">\n        <meta name="viewport" content="width=device-width, initial-scale=1.0, shrink-to-fit=no">\n        <link href="https://fonts.googleapis.com/css?family=DM+Sans&display=swap" rel="stylesheet">            <link rel="stylesheet" media="all" href="https://bib.cnrs.fr/wp-content/themes/portail/style.css?v=2.2" type="text/css"><link rel=\'dns-prefetch\' href=\'//bib.cnrs.fr\' />\n<link rel=\'dns-prefetch\' href=\'//s.w.org\' />\n<link rel="alternate" type="application/rss+xml" title="BibCnrs &raquo; Home Comments Feed" href="https://bib.cnrs.fr/home/feed/" />\n\t\t<script type="text/javascript">\n\t\t\twindow._wpemojiSettings = {"baseUrl":"https:\\/\\/s.w.org\\/images\\/core\\/emoji\\/13.0.0\\/72x72\\/","ext":".png","

In [3]:
parser = "html.parser"
soup = BeautifulSoup(x, parser)
print(soup)

<html>
<head>
<title>BibCnrs</title>
<meta charset="utf-8"/>
<meta content="Portail BibCnrs IST" name="description"/>
<meta content="yes" name="apple-mobile-web-app-capable">
<meta content="index follow" name="robots"/>
<meta content="width=device-width, initial-scale=1.0, shrink-to-fit=no" name="viewport"/>
<link href="https://fonts.googleapis.com/css?family=DM+Sans&amp;display=swap" rel="stylesheet"/> <link href="https://bib.cnrs.fr/wp-content/themes/portail/style.css?v=2.2" media="all" rel="stylesheet" type="text/css"/><link href="//bib.cnrs.fr" rel="dns-prefetch">
<link href="//s.w.org" rel="dns-prefetch">
<link href="https://bib.cnrs.fr/home/feed/" rel="alternate" title="BibCnrs » Home Comments Feed" type="application/rss+xml"/>
<script type="text/javascript">
			window._wpemojiSettings = {"baseUrl":"https:\/\/s.w.org\/images\/core\/emoji\/13.0.0\/72x72\/","ext":".png","svgUrl":"https:\/\/s.w.org\/images\/core\/emoji\/13.0.0\/svg\/","svgExt":".svg","source":{"concatemoji":"https:\

## Parsing directly on CNRS database looks a bit complicated

For now we'll just try parsing the Arxiv website and see how it goes

In [4]:
arxiv_url = "https://arxiv.org/search/?query={}&searchtype=title&abstracts=show&order=-announced_date_first&size=50&start={}"


In [8]:
# choose the query we want
# careful: here we're just working on the first 50 results (the website presents them 50 by 50)
# if we want more we just have to iterate on size adding 50 by 50


            
def get_pdf_urls(website_url, query, total_size):
    """
    ARGS: 
         - total_size is the total amount of articles we have on the web page. we have to check the webpage beforehand
        to know what it amounts to
    OUTPUT: a list of urls linking to the articles
    """
    urls = []
    for i in range (0, total_size, 50):
        size = i

        x = requests.get(website_url.format(query, size)).content
        parser = "html.parser"
        soup = BeautifulSoup(x, parser)

        results = soup("p", class_="list-title is-inline-block")


        for result in results:
            for a in result.find_all('a', href=True):
                if "pdf" in a['href']:
                    urls.append(a['href']+".pdf")
    return urls

urls = get_pdf_urls(arxiv_url, "machine+learning", 50)
print(urls)
print(type(urls[0]))

['https://arxiv.org/pdf/2010.02866.pdf', 'https://arxiv.org/pdf/2010.02749.pdf', 'https://arxiv.org/pdf/2010.02715.pdf', 'https://arxiv.org/pdf/2010.02670.pdf', 'https://arxiv.org/pdf/2010.02576.pdf', 'https://arxiv.org/pdf/2010.02523.pdf', 'https://arxiv.org/pdf/2010.02374.pdf', 'https://arxiv.org/pdf/2010.02317.pdf', 'https://arxiv.org/pdf/2010.02213.pdf', 'https://arxiv.org/pdf/2010.02174.pdf', 'https://arxiv.org/pdf/2010.02087.pdf', 'https://arxiv.org/pdf/2010.02086.pdf', 'https://arxiv.org/pdf/2010.02011.pdf', 'https://arxiv.org/pdf/2010.01996.pdf', 'https://arxiv.org/pdf/2010.01976.pdf', 'https://arxiv.org/pdf/2010.01968.pdf', 'https://arxiv.org/pdf/2010.01711.pdf', 'https://arxiv.org/pdf/2010.01709.pdf', 'https://arxiv.org/pdf/2010.01668.pdf', 'https://arxiv.org/pdf/2010.01582.pdf', 'https://arxiv.org/pdf/2010.01431.pdf', 'https://arxiv.org/pdf/2010.01213.pdf', 'https://arxiv.org/pdf/2010.01163.pdf', 'https://arxiv.org/pdf/2010.01149.pdf', 'https://arxiv.org/pdf/2010.01030.pdf',

### Getting the text

Now that we have the pdf url we have two options: either use beautiful soup, see the web page as a html doc and get the texts from there; either download completely the pdf and use pdfplumber or something alike.

In [2]:
# We'll try with beautiful soup

x = requests.get("https://arxiv.org/pdf/2010.02866.pdf").content
parser = "html.parser"
soup = BeautifulSoup(x, parser)
print(soup)
#results = soup.findall("span", string = True, limit = 5)
#print(results)




Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Problems with scrapping

We'll try downloading it here first and then getting the text out

In [5]:
destination = "/home/martin/Desktop/ImpAgt/test.pdf"

chunk_size = 4000

import requests

url = "https://arxiv.org/pdf/2010.02866.pdf"
r = requests.get(url, stream=True)

with open(destination, 'wb') as fd:
    for chunk in r.iter_content(chunk_size):
        fd.write(chunk)
        


Notes:

 - On bloque un peu dans notre scrapping sur la base de recherche du CNRS. On pense que c'est parce que le site est en dynamique donc il faut qu'on creuse un peu 
 - on est parti sur un site plus simple: Arxiv
 - On arrive pas a récupérer directement les docs donc on passe par un chemin détourné: on download en bloc et on utililse un package pour ca